In [1]:
from synth import Osc, Synth, Tone
from dissonance_calculator import dissonance_total
import plotly.graph_objects as go
import numpy as np
import os

# Initialize parameters

In [2]:
# number of overtones to calculate
NUM_OVERS = 10
# base note of the intervals to calculate the dissonance of
BASE_NOTE = "C4"
# convert to fre
base_freq = Tone.from_name(BASE_NOTE).freq
# base pressure of the fundamental
BASE_P = 0.02
# resolution of the graphs
NX = 100
NY = 100


## Create the sound generator

In [3]:
# Simple sine with multiple voices, one is an octave lower
a = Osc.sine_osc(voices=3, detune=0.05,p=BASE_P, phase_random_range=0)
b = Osc.sine_osc(pitch=-12, p=BASE_P * 0.5)
SineSynth = Synth([a, b])


In [4]:
# Saw sound
def saw(i, n):
    return (-1 ** n) * i / n
a = Osc(decay=saw, num_overs=NUM_OVERS, p=BASE_P, phase_random_range=0)
SawSynth = Synth([a])


### Test the sounds

In [5]:
SineSynth.play(["C4", "E4"])

In [6]:
SawSynth.play(["C4", "E4"])

### Visualize the sound being made

In [7]:
secs = 0.05
rate = 44100
S = SawSynth
notes = ["C4", "E4"]

tones = S.get_tones(notes)
ts = np.linspace(0.0, secs, int(rate * secs))
data = np.sum( 
    [
        tone.p * (2 ** 0.5) * np.sin(2 * np.pi * tone.freq * ts + tone.phase)
        for tone in tones
    ],
    axis=0,
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=ts, y=data))
fig.update_xaxes(title_text='Time (s)')
fig.update_yaxes(title_text='Pressure (Pa)')
fig.show()


# Graph the dissonance over intervals from base frequency

In [8]:
# Interval names and values
intervals = ["P1", "m2", "M2", "m3", "M3", "P4", "TT", "P5", "m6", "M6", "m7", "M7", "P8", "m9", "M9"]
interval_vals = [round(base_freq * 2 ** (i / 12), 3) for i in range(len(intervals))]

In [18]:
# choose the synth
S = SawSynth

# axes mesh
xs = np.linspace(base_freq, base_freq * 2.5, NX)
tones = [S.get_tones([base_freq, f]) for f in xs]
ys = [dissonance_total([t.freq for t in tone], [t.p for t in tone]) for tone in tones]

# make graphs
fig = go.Figure()
fig.add_trace(go.Scatter(x=xs, y=ys, mode="lines"))

# make it pretty
fig.update_layout(width=1600, height=500)
fig.update_xaxes(
    ticktext=intervals,
    tickvals=interval_vals,
    title_text="Intervals",
    type="log"
)
fig.update_yaxes(title_text="Dissonance")

fig.show()


In [19]:
# saving the graph as png
if not os.path.exists("images"):
    os.mkdir("images")
fig.write_image(f"images/intervals from {BASE_NOTE}.png")

# Graph all the triads' dissonance 
*(the triads' root note will have a frequency of base_freq)*  
*takes around 2 minutes if*
$NX * NY * NUM\_ OVERS^2 = 100 * 100 * 10^2$

In [24]:
# Dissonance calculation
# will be more efficient if we only calculate half of the possible intervals since it's symmetrical

# axes mesh
xs = np.linspace(base_freq, base_freq * 2.5, NX)
ys = np.linspace(base_freq, base_freq * 2.5, NY)
zs = np.zeros((NX, NY))

# calculate dissonance
for i in range(NX):
    for j in range(NY):
        tones = S.get_tones([base_freq, xs[i], ys[j]])
        zs[i, j] = dissonance_total([tone.freq for tone in tones], [tone.p for tone in tones])


## 3D graph

In [61]:
fig = go.Figure()
fig.add_trace(go.Surface(z=zs, x=xs, y=ys))

fig.update_layout(
    scene = dict(
        xaxis = dict(
            ticktext=intervals,
            tickvals=interval_vals,
            title_text="Interval",
            type="log"
        ),
        yaxis = dict(
            ticktext=intervals,
            tickvals=interval_vals,
            title_text="Interval",
            type="log"
        ),
        zaxis = dict(showticklabels=False, title="Dissonance"),
    ),
    scene_camera = dict(
        eye=dict(x=1.5, y=1.25, z=1),
        center=dict(x=0, y=0, z=-0.2)
    ),
    width=800, height=800
)
fig.show()



In [62]:
# saving the graph as png
if not os.path.exists("images"):
    os.mkdir("images")
fig.write_image(f"images/{BASE_NOTE} triads.png")

# 2D Heatmap

In [65]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=zs, x=xs, y=ys,))

fig.update_layout(width=800, height=800)
fig.update_xaxes(ticktext=intervals,
    tickvals=interval_vals,
    type="log"
)
fig.update_yaxes(
    ticktext=intervals,
    tickvals=interval_vals,
    type="log"
)
fig.show()


In [66]:
# saving the graph as png
if not os.path.exists("images"):
    os.mkdir("images")
fig.write_image(f"images/{BASE_NOTE} triads heatmap.png")